In [3]:
import pandas as pd
import os
import requests
import json
from functions import download
import xml.etree.ElementTree as et
import numpy as np

In [5]:

response = requests.get("https://datos.gob.mx/busca/api/3/action/package_search?q=gasolina")
dataDownload = response.json()
dataDownload

{'help': 'https://datos.gob.mx/busca/api/3/action/help_show?name=package_search',
 'success': True,
 'result': {'count': 23,
  'sort': 'score desc, metadata_modified desc',
  'facets': {},
  'results': [{'license_title': None,
    'maintainer': None,
    'relationships_as_object': [],
    'private': False,
    'maintainer_email': None,
    'num_tags': 7,
    'id': '0bfa4e75-4e06-4648-b5ba-f245d2844dd6',
    'metadata_created': '2019-06-27T23:49:09.316620',
    'metadata_modified': '2021-01-13T17:49:22.768112',
    'author': None,
    'author_email': None,
    'state': 'active',
    'version': None,
    'creator_user_id': '868b5c2e-41f1-424c-9ea2-ac3ca3da5768',
    'type': 'dataset',
    'resources': [{'mimetype': None,
      'cache_url': None,
      'hash': '',
      'description': 'Registro de estaciones de servicio (gasolineras) en formato XML; incluye el id de la estación, el nombre de la empresa, la dirección y las coordenadas geográficas.',
      'name': 'Listado de Estaciones de 

In [7]:
urlLinkLocacion = dataDownload["result"]["results"][0]["resources"][0]["url"]
urlLinkPrecios = dataDownload["result"]["results"][0]["resources"][1]["url"]

pwd = os.getcwd()
download(urlLinkPrecios, pwd + 'Precios.xml')
download(urlLinkLocacion, pwd +'Lugar.xml')


In [10]:
df_cols = ["place_id", "gas_type", "gas_price"]
rows = []

xtree = et.parse(pwd + '/Precios.xml')
xroot = xtree.getroot() # Gets </places> tag as root

for node in xroot:
    placeId = node.attrib.get("place_id")  #Gets <place> attribute "place_id"
    for inside in node:
        gas_type = inside.attrib.get("type") #Gets <gas_price> attribute "type"
        gas_price = inside.text #Gets <gas_price> text "gas_price"
        rows.append({"place_id": placeId, "gas_type": gas_type, 
                 "gas_price": gas_price})

prices_df = pd.DataFrame(rows, columns= df_cols)
prices_df

,place_id,gas_type,gas_price
0,11703,regular,20.79
1,11703,premium,22.79
2,11702,regular,20.14
3,11702,premium,23.35
4,11702,diesel,21.5
...,...,...,...
33732,11704,premium,22.49
33733,11704,diesel,21.49
33734,2807,regular,19.98
33735,2807,diesel,21.99


In [13]:
df_cols = ["place_id", "location_name", "latitude", "longitude"]
rows = []

xtree = et.parse(pwd + '/Lugar.xml')
xroot = xtree.getroot() # Gets </places> tag as root

for node in xroot:
    placeId = node.attrib.get("place_id")
    location_name = node.find("name").text
    for inside in node.iter("location"):
        latitude = inside.find("x").text
        longitude = inside.find("y").text

        rows.append({"place_id": placeId, "location_name": location_name, 
                "latitude": latitude, "longitude": longitude})

locations_df = pd.DataFrame(rows, columns= df_cols)
locations_df

,place_id,location_name,latitude,longitude
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",-116.9214,32.47641
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",-99.74484,20.3037
2,2041,"DIAZ GAS, S.A. DE C.V.",-106.4514,31.71947
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",-103.3042,20.71413
4,2043,"PETROMAX, S.A. DE C.V.",-98.29977,26.03787
...,...,...,...,...
12890,27593,"SERVICIO EL LEONCITO, S.A. DE C.V.",-100.9331,21.81464
12891,27597,PRO M3 INFRAESTRUCTURA S.A. DE C.V.,-100.4573,20.82254
12892,27598,Servicio Cupula SA de CV,-98.91469,19.41953
12893,27599,"SERVICIO FACIL DEL SURESTE, S.A. DE C.V.",-94.95576,17.89041


In [16]:
locations_df.isnull().sum()


place_id         0
location_name    0
latitude         0
longitude        0
dtype: int64

In [18]:
prices_df.isnull().sum()

place_id     0
gas_type     0
gas_price    0
dtype: int64

In [21]:
gas_table = locations_df.merge(prices_df, how="left", on="place_id")
gas_table

,place_id,location_name,latitude,longitude,gas_type,gas_price
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",-116.9214,32.47641,regular,19.99
1,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",-116.9214,32.47641,premium,22.99
2,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",-116.9214,32.47641,diesel,20.59
3,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",-99.74484,20.3037,premium,22.15
4,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",-99.74484,20.3037,diesel,21.61
...,...,...,...,...,...,...
33980,27599,"SERVICIO FACIL DEL SURESTE, S.A. DE C.V.",-94.95576,17.89041,regular,19.99
33981,27599,"SERVICIO FACIL DEL SURESTE, S.A. DE C.V.",-94.95576,17.89041,premium,21.55
33982,27599,"SERVICIO FACIL DEL SURESTE, S.A. DE C.V.",-94.95576,17.89041,diesel,21.19
33983,27600,"SERVICIO FACIL DEL SURESTE, S.A. DE C.V.",-93.04568,17.98012,regular,20.04


In [25]:
gas_table.isnull().sum()

place_id           0
location_name      0
latitude           0
longitude          0
gas_type         248
gas_price        248
dtype: int64

In [26]:
gas_table_modified = gas_table.copy()

In [29]:
gas_table_modified.groupby(['gas_type']).size()

gas_type
diesel      9409
premium    11745
regular    12583
dtype: int64

In [31]:
gas_table_pivot = gas_table_modified.pivot(index=['place_id', 'location_name', "latitude", "longitude"], columns='gas_type', values="gas_price").reset_index()
gas_table_pivot




gas_type,place_id,location_name,latitude,longitude,NaN,diesel,premium,regular
0,10000,"SERVICIOS GASOLINEROS DE MEXICO, S.A. DE C.V.",-99.17684,23.75344,NaN,21.99,23.5,19.98
1,10001,SERVICIOS PALO VERDES SA DE CV,-109.9156,27.46514,NaN,22.1,22.99,19.38
2,10002,ELIO LOPEZ DOMINGUEZ,-97.64679,19.13942,NaN,16.79,17.6,15.87
3,10003,SERVICIO LINDA VISTA SA DE CV,-106.4816,31.69662,NaN,NaN,NaN,15.2
4,10004,ALEJANDRO MANUEL MENDEZ HURTADO,-102.0301,19.40397,NaN,21.83,21.99,19.89
...,...,...,...,...,...,...,...,...
12890,9995,"SERVICIOS GASOLINEROS DE MEXICO, S.A. DE C.V.",-100.3123,25.68669,NaN,NaN,23.79,21.93
12891,9996,"SERVICIOS GASOLINEROS DE MEXICO, S.A. DE C.V.",-100.1816,25.70952,NaN,21.14,22.79,20.68
12892,9997,ENERGETICOS LA VALENTINA SA DE CV,-103.585,19.88209,NaN,21.89,22.29,20.49
12893,9998,"SERVICIOS GASOLINEROS DE MEXICO, S.A. DE C.V.",-100.4493,25.65615,NaN,NaN,23.75,21.88


In [35]:
gas_table_pivot = gas_table_pivot[['place_id','location_name','latitude','longitude','diesel','premium','regular']]

In [45]:
gas_table_pivot.to_csv(pwd + '/mergeLocations.csv')